Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset_raw = save['train_dataset']
  train_labels_raw = save['train_labels']
  valid_dataset_raw = save['valid_dataset']
  valid_labels_raw = save['valid_labels']
  test_dataset_raw = save['test_dataset']
  test_labels_raw = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset_raw.shape, train_labels_raw.shape)
  print('Validation set', valid_dataset_raw.shape, valid_labels_raw.shape)
  print('Test set', test_dataset_raw.shape, test_labels_raw.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset_raw, train_labels_raw)
valid_dataset, valid_labels = reformat(valid_dataset_raw, valid_labels_raw)
test_dataset, test_labels = reformat(test_dataset_raw, test_labels_raw)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Logistic regression with regularization

In [41]:
train_subset = 10000
graph = tf.Graph()

with graph.as_default():
    
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    regularization_coef = tf.constant(0.2, dtype=tf.float32)
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + regularization_coef * tf.nn.l2_loss(weights)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [42]:
num_steps = 801

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 617.104797
Training accuracy: 10.0%
Validation accuracy: 11.9%
Loss at step 100: 1.141497
Training accuracy: 80.5%
Validation accuracy: 80.8%
Loss at step 200: 1.147264
Training accuracy: 79.2%
Validation accuracy: 80.5%
Loss at step 300: 1.145424
Training accuracy: 79.4%
Validation accuracy: 80.6%
Loss at step 400: 1.144738
Training accuracy: 79.4%
Validation accuracy: 80.7%
Loss at step 500: 1.144482
Training accuracy: 79.5%
Validation accuracy: 80.7%
Loss at step 600: 1.144384
Training accuracy: 79.5%
Validation accuracy: 80.7%
Loss at step 700: 1.144344
Training accuracy: 79.5%
Validation accuracy: 80.7%
Loss at step 800: 1.144328
Training accuracy: 79.5%
Validation accuracy: 80.7%
Test accuracy: 86.4%


Neural Network with regularization

In [59]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    regularization_coef = tf.constant(0.001, dtype=tf.float32)
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) 
            + regularization_coef * tf.nn.l2_loss(weights))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [62]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 18.987785
Minibatch accuracy: 10.9%
Validation accuracy: 14.3%
Minibatch loss at step 500: 2.757422
Minibatch accuracy: 79.7%
Validation accuracy: 76.3%
Minibatch loss at step 1000: 1.649225
Minibatch accuracy: 82.0%
Validation accuracy: 79.1%
Minibatch loss at step 1500: 1.021269
Minibatch accuracy: 87.5%
Validation accuracy: 80.7%
Minibatch loss at step 2000: 1.093402
Minibatch accuracy: 78.9%
Validation accuracy: 81.1%
Minibatch loss at step 2500: 0.797037
Minibatch accuracy: 87.5%
Validation accuracy: 82.5%
Minibatch loss at step 3000: 0.886618
Minibatch accuracy: 78.1%
Validation accuracy: 82.3%
Test accuracy: 88.5%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [71]:
batch_size = 3

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    regularization_coef = tf.constant(0.001, dtype=tf.float32)
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) 
            + regularization_coef * tf.nn.l2_loss(weights))
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [72]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.519773
Minibatch accuracy: 0.0%
Validation accuracy: 18.0%
Minibatch loss at step 500: 2.379991
Minibatch accuracy: 100.0%
Validation accuracy: 71.5%
Minibatch loss at step 1000: 9.358800
Minibatch accuracy: 66.7%
Validation accuracy: 72.5%
Minibatch loss at step 1500: 9.492346
Minibatch accuracy: 66.7%
Validation accuracy: 72.2%
Minibatch loss at step 2000: 1.372003
Minibatch accuracy: 100.0%
Validation accuracy: 71.5%
Minibatch loss at step 2500: 20.389418
Minibatch accuracy: 33.3%
Validation accuracy: 71.4%
Minibatch loss at step 3000: 33.196548
Minibatch accuracy: 0.0%
Validation accuracy: 64.2%
Test accuracy: 69.3%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [11]:
batch_size = 256

neurons_amount = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, neurons_amount]))
  biases1 = tf.Variable(tf.zeros([neurons_amount]))

  weights2 = tf.Variable(
    tf.truncated_normal([neurons_amount, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  layer1 = tf.matmul(tf_train_dataset, weights1) + biases1
  layer2 = tf.nn.relu(tf.nn.dropout(layer1, 0.75))
  layer3 = tf.matmul(layer2, weights2) + biases2
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=layer3))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), weights2) + biases2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [12]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 384.157227
Minibatch accuracy: 9.0%
Validation accuracy: 32.6%
Minibatch loss at step 500: 9.901791
Minibatch accuracy: 83.2%
Validation accuracy: 80.4%
Minibatch loss at step 1000: 8.609006
Minibatch accuracy: 84.4%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 7.696097
Minibatch accuracy: 81.2%
Validation accuracy: 82.9%
Minibatch loss at step 2000: 2.857599
Minibatch accuracy: 87.1%
Validation accuracy: 79.9%
Minibatch loss at step 2500: 2.837073
Minibatch accuracy: 88.3%
Validation accuracy: 80.2%
Minibatch loss at step 3000: 3.504191
Minibatch accuracy: 80.9%
Validation accuracy: 83.2%
Minibatch loss at step 3500: 3.990180
Minibatch accuracy: 84.0%
Validation accuracy: 83.2%
Minibatch loss at step 4000: 3.570932
Minibatch accuracy: 84.4%
Validation accuracy: 82.2%
Minibatch loss at step 4500: 1.893798
Minibatch accuracy: 86.7%
Validation accuracy: 83.7%
Minibatch loss at step 5000: 3.265713
Minibatch accuracy: 82.8%
Validation accura

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [44]:
batch_size = 256

neurons_amount = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, neurons_amount]))
  biases1 = tf.Variable(tf.zeros([neurons_amount]))

  weights2 = tf.Variable(
    tf.truncated_normal([neurons_amount, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 1, 1)

  # Training computation.
  layer1 = tf.matmul(tf_train_dataset, weights1) + biases1
  layer2 = tf.nn.relu(tf.nn.dropout(layer1, 0.75))
  layer3 = tf.matmul(layer2, weights2) + biases2
  
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=layer3))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), weights2) + biases2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [45]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 489.998688
Minibatch accuracy: 5.1%
Validation accuracy: 36.6%
Minibatch loss at step 500: 18.488340
Minibatch accuracy: 79.7%
Validation accuracy: 79.4%
Minibatch loss at step 1000: 10.436325
Minibatch accuracy: 84.4%
Validation accuracy: 81.9%
Minibatch loss at step 1500: 7.303227
Minibatch accuracy: 82.8%
Validation accuracy: 82.0%
Minibatch loss at step 2000: 3.138728
Minibatch accuracy: 83.6%
Validation accuracy: 82.0%
Minibatch loss at step 2500: 3.489586
Minibatch accuracy: 88.3%
Validation accuracy: 81.4%
Minibatch loss at step 3000: 3.597244
Minibatch accuracy: 80.5%
Validation accuracy: 82.9%
Minibatch loss at step 3500: 6.953432
Minibatch accuracy: 82.8%
Validation accuracy: 78.7%
Minibatch loss at step 4000: 3.032752
Minibatch accuracy: 84.4%
Validation accuracy: 81.7%
Minibatch loss at step 4500: 2.133861
Minibatch accuracy: 84.8%
Validation accuracy: 84.0%
Minibatch loss at step 5000: 2.398272
Minibatch accuracy: 82.8%
Validation accu